## Sentiment Analysis

In [1]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import re
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed

prefix = 'review_topics' #Replace with the prefix under which you want to store the data if needed

arn:aws:iam::443501626368:role/service-role/AmazonSageMaker-ExecutionRole-20200806T142735


In [3]:
print(bucket)

sagemaker-us-east-1-443501626368


### Preprocess training and test data

In [4]:
# define a simple tokenizer (NLTK won't be available to us later on, in our Lambda function)

def simple_tokenizer(input_text):
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(=)|(`)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|(\n)|(\t)")
    tokens = REPLACE_NO_SPACE.sub("", input_text.lower())
    tokens = REPLACE_WITH_SPACE.sub(" ", tokens) # note that blazing text expects space-separated tokens
    return tokens

In [32]:
# read and process the dataset
def preprocesser(input_file, output_file, output_csv):
    
    # read in the dataset
    filepath=Path.joinpath(Path.cwd().parent, 'data', 'amazon_review_polarity_csv', input_file)
    df = pd.read_csv(filepath, names=["Label", "Title", "Review"])
    df = df.sample(frac = 0.35)
    # class 0 is the negative and class 1 is the positive
    df['Label']=df['Label']-1
    
    # drop Title, focus on review itself
    df=df.drop('Title', axis=1)
    
    # Shuffle reviews and corresponding labels within training and test sets
    df = df.sample(frac = 1)
    
    # apply the simple tokenizer
    df['Review']=df['Review'].apply(simple_tokenizer)
    
    # Prefix the index-ed label with __label__
    df['Label']=df['Label'].apply(lambda row: "__label__" +  str(row) )
    
    # convert the transformed dataframe into a list
    transformed_rows = np.array(df).tolist()
    
    # write to csv file (for blazingtext)
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n') # notice the delimiter.
        csv_writer.writerows(transformed_rows)
        
    # write to csv file (for LDA topic modeling)
    df.to_csv(output_csv, index=False)

In [33]:
%%time

# Preparing the training dataset
preprocesser('train.csv', 'polarity.train', 'train_processed.csv')

CPU times: user 3min 50s, sys: 2.03 s, total: 3min 52s
Wall time: 3min 56s


In [8]:
%%time

# Preparing the validation dataset        
preprocesser('test.csv', 'polarity.validation',  'test_processed.csv')

CPU times: user 17.6 s, sys: 164 ms, total: 17.8 s
Wall time: 18.6 s


The data preprocessing cell might take a minute to run. After the data preprocessing is complete, we need to upload it to S3 so that it can be consumed by SageMaker to execute training jobs. We'll use Python SDK to upload these two files to the bucket and prefix location that we have set above.

In [9]:
%%time

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

sess.upload_data(path='polarity.train', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='polarity.validation', bucket=bucket, key_prefix=validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

CPU times: user 7.08 s, sys: 909 ms, total: 7.99 s
Wall time: 5.55 s


Next we need to setup an output location at S3, where the model artifact will be dumped. These artifacts are also the output of the algorithm's traning job.

In [10]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

## Training the Sentiment Classifier
Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.

In [11]:
region_name = boto3.Session().region_name

In [12]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Using SageMaker BlazingText container: 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:latest (us-east-1)


Now, let's define the SageMaker `Estimator` with resource configurations and hyperparameters to train Text Classification on *DBPedia* dataset, using "supervised" mode on a `c4.4xlarge` instance.


In [13]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


Please refer to [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) for the complete list of hyperparameters.

In [14]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

Now that the hyper-parameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm consumes.

In [15]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the Estimator classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins.

In [16]:
bt_model.fit(inputs=data_channels, logs=True)

2020-08-13 16:48:57 Starting - Starting the training job...
2020-08-13 16:49:02 Starting - Launching requested ML instances.........
2020-08-13 16:50:43 Starting - Preparing the instances for training...
2020-08-13 16:51:27 Downloading - Downloading input data...
2020-08-13 16:51:49 Training - Downloading the training image...
2020-08-13 16:52:06 Training - Training image download completed. Training in progress.Arguments: train
[08/13/2020 16:52:07 WARNING 140211784693568] Loggers have already been setup.
[08/13/2020 16:52:07 WARNING 140211784693568] Loggers have already been setup.
[08/13/2020 16:52:07 INFO 140211784693568] nvidia-smi took: 0.0252530574799 secs to identify 0 gpus
[08/13/2020 16:52:07 INFO 140211784693568] Running single machine CPU BlazingText training using supervised mode.
[08/13/2020 16:52:07 INFO 140211784693568] Processing /opt/ml/input/data/train/polarity.train . File size: 349 MB
[08/13/2020 16:52:07 INFO 140211784693568] Processing /opt/ml/input/data/validati

## Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. 
note on pricing: https://aws.amazon.com/sagemaker/pricing/instance-types/

In [17]:
text_classifier = bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!

BlazingText supports `application/json` as the content-type for inference. The payload should contain a list of sentences with the key as "**instances**" while being passed to the endpoint.

In [19]:
sentences = ['best happy joy smile wonderful amazing loved it recommend very glad excited eager feeling good quality top-notch great']
tokenized_sentences = [simple_tokenizer(sentences[0])]
payload = {"instances" : tokenized_sentences}
response = text_classifier.predict(json.dumps(payload)) 
predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

[
  {
    "prob": [
      1.0000100135803223
    ],
    "label": [
      "__label__1"
    ]
  }
]


### Prepare for Lambda Function
When we eventually construct our Lambda function we won't have access to the `text_classifier` object, so how do we call a SageMaker endpoint?

Python functions that are used in Lambda have access to another Amazon library called boto3. The `boto3` library provides an API for working with Amazon services, including SageMaker. To start with, we need to get a handle to the SageMaker runtime.

In [20]:
import boto3
runtime = boto3.Session().client('sagemaker-runtime')
print(runtime)

And now that we have access to the SageMaker runtime, we can ask it to make use of (invoke) an endpoint that has already been created. However, we need to provide SageMaker with the name of the deployed endpoint. To find this out we can print it out using the text_classifier object.

In [21]:
print(text_classifier.endpoint)

blazingtext-2020-08-13-16-48-57-681


In [28]:
# write function to call the endpoint
def make_prediction(sentences):
    sentences=[sentences]
    tokenized_sentences = [simple_tokenizer(sentences[0])]
    payload = {"instances" : tokenized_sentences}
    response = text_classifier.predict(json.dumps(payload)) 

    response = runtime.invoke_endpoint(EndpointName = text_classifier.endpoint, # The name of the endpoint we created
                                           ContentType = 'application/json', # The data format that is expected
                                           Body = json.dumps(payload))

    output = json.loads(response['Body'].read().decode('utf-8'))
    print(output[0]['prob'][0])
    print(output[0]['label'][0].split('__label__')[1])

In [30]:
# try a positive sentence

make_prediction('best happy joy smile wonderful amazing loved it recommend very glad excited eager feeling good quality top-notch great')

1.0000100135803223
1


In [31]:
# try a negative sentence

make_prediction('awful horrible hated worst never disgusting bad terrible destroy bad')

1.0000100135803223
0
